<a href="https://colab.research.google.com/github/81300/AI-Notebooks/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM text generation notebook for Google Colab

Version: GPU

This notebook uses **oobabooga**'s [text-generation-webui project](https://github.com/oobabooga/text-generation-webui).

Run all the cells and a public URL will appear at the bottom.

## Useful links

* [Web UI wiki](https://github.com/oobabooga/text-generation-webui/wiki)
* [Hugging Face model hub](https://huggingface.co/models)
* [Chatbot character creator](https://oobabooga.github.io/character-creator.html)

## Credits

* [oobabooga's notebook](https://github.com/oobabooga/AI-Notebooks)
* [text-generation-webui](https://github.com/oobabooga/text-generation-webui)

In [ ]:
#@title 1. Keep the tab alive { display-mode: "form" }

#@markdown This can prevent Colab from terminating your runtime. Press play on the music player that will appear below:

%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
#@title 2. Install the web UI

Save_data_to_Google_Drive = False #@param {type:"boolean"}
Install_GPTQ = True #@param {type:"boolean"}
Reinstall_all = False #@param {type:"boolean"}

if Save_data_to_Google_Drive:
  import os
  import shutil
  from google.colab import drive
  drive.mount("/content/drive")
  drive_folder = "/content/drive/MyDrive"

if Reinstall_all:
  !rm -r $repo_path

import os
repo_path = "/content/Colab-TextGen"
if not os.path.exists(repo_path):
  os.mkdir(repo_path)

%cd $repo_path
![[ ! -d .git ]] && git clone https://github.com/oobabooga/text-generation-webui .
!git reset --hard
!git pull
!pip install --no-cache-dir --progress-bar=off -r requirements.txt
!pip install --no-cache-dir --progress-bar=off -r extensions/api/requirements.txt
!wget https://oobabooga.github.io/settings-colab.json -O settings-colab.json

if Save_data_to_Google_Drive:
  if not os.path.exists(f"{drive_folder}/oobabooga-data"):
    os.mkdir(f"{drive_folder}/oobabooga-data")
  if not os.path.exists(f"{drive_folder}/oobabooga-data/logs"):
    os.mkdir(f"{drive_folder}/oobabooga-data/logs")
  if not os.path.exists(f"{drive_folder}/oobabooga-data/softprompts"):
    os.mkdir(f"{drive_folder}/oobabooga-data/softprompts")
  if not os.path.exists(f"{drive_folder}/oobabooga-data/characters"):
    shutil.move(f"{repo_path}/characters", f"{drive_folder}/oobabooga-data/characters")
  else:
    !rm -r "$repo_path/characters"
    
  !rm -r "$repo_path/softprompts"
  !ln -sf "$drive_folder/oobabooga-data/logs" "$repo_path/logs"
  !ln -sf "$drive_folder/oobabooga-data/softprompts" "$repo_path/softprompts"
  !ln -sf "$drive_folder/oobabooga-data/characters" "$repo_path/characters"

if Install_GPTQ:
  !mkdir repositories
  %cd repositories
  !git clone https://github.com/oobabooga/GPTQ-for-LLaMa.git -b cuda
  %cd GPTQ-for-LLaMa
  !python setup_cuda.py install

In [ ]:
#@title 3. Select a model

HuggingFace_Organization = "" #@param {type:"string"}
HuggingFace_Model = "" #@param {type:"string"}
HuggingFace_Branch = "main" #@param ["main"] {allow-input: true}
Redownload = False #@param {type: "boolean"}
Delete_all_models = False #@param {type: "boolean"}

repo_path = "/content/Colab-TextGen"
models_path = "/content/Colab-TextGen/models"

if HuggingFace_Branch != "main":
  model_name = HuggingFace_Organization + "_" + HuggingFace_Model + "_" + HuggingFace_Branch
else:
  model_name = HuggingFace_Organization + "_" + HuggingFace_Model

if Delete_all_models:
  import os
  if os.path.isdir(models_path):
    %cd $models_path
    !find . -mindepth 1 -maxdepth 1 -type d -exec rm -rv "{}" \;
  else:
    print("Not a valid models directory.")

%cd $repo_path
if Redownload:
  !python download-model.py $HuggingFace_Organization/$HuggingFace_Model --branch $HuggingFace_Branch --clean
else:
  !python download-model.py $HuggingFace_Organization/$HuggingFace_Model --branch $HuggingFace_Branch

In [ ]:
#@title 4. Launch

Mode = "Notebook" #@param ["Notebook", "Chat", "API"] {allow-input: false}
Extra_launch_arguments = None #@param ["--load-in-8bit", "--wbits 4 --model_type llama --groupsize 128", "--wbits 4 --model_type llama --groupsize 32"] {allow-input: true}

if Extra_launch_arguments is None:
  Extra_launch_arguments = ""

%cd $repo_path
if Mode == "Notebook":
  !python server.py --share --model $model_name --settings settings-colab.json --notebook $Extra_launch_arguments
elif Mode == "Chat":
  !python server.py --share --model $model_name --settings settings-colab.json --chat $Extra_launch_arguments
elif Mode == "API":
  !python server.py --share --model $model_name --extensions api $Extra_launch_arguments
else:
  print("No valid mode selected.")